In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import imageio.v2 as imageio
from tqdm.auto import tqdm
from concurrent.futures import ProcessPoolExecutor

input_path = '/kaggle/input/2024-flame-ai-challenge/dataset/'

train_df = pd.read_csv(os.path.join(input_path,'train.csv'))
train_df.head()

,id,u,alpha,Nt,Nx,Ny,theta_filename,ustar_filename,xi_filename
0,804025,2,10,150,113,32,theta_K_id804025.dat,ustar_ms-1_id804025.dat,xi_id804025.dat
1,875935,2,5,150,113,32,theta_K_id875935.dat,ustar_ms-1_id875935.dat,xi_id875935.dat
2,930086,6,0,150,113,32,theta_K_id930086.dat,ustar_ms-1_id930086.dat,xi_id930086.dat
3,661713,4,5,150,113,32,theta_K_id661713.dat,ustar_ms-1_id661713.dat,xi_id661713.dat
4,633229,2,0,150,113,32,theta_K_id633229.dat,ustar_ms-1_id633229.dat,xi_id633229.dat


In [2]:
def load_dataX(idx):
    csv_file = train_df.reset_index().to_dict(orient='list')
    dir_path = os.path.join(input_path, "train")
    id = csv_file['id'][idx]
    nt, Nx, Ny = csv_file['Nt'][idx], csv_file['Nx'][idx], csv_file['Ny'][idx]
    theta = np.fromfile(os.path.join(dir_path, csv_file['theta_filename'][idx]), dtype="<f4").reshape(nt, Nx, Ny)
    ustar = np.fromfile(os.path.join(dir_path, csv_file['ustar_filename'][idx]), dtype="<f4").reshape(nt, Nx, Ny)
    xi_f = np.fromfile(os.path.join(dir_path, csv_file['xi_filename'][idx]), dtype="<f4").reshape(nt, Nx, Ny)
    uin  = csv_file['u'][idx]
    alpha = csv_file['alpha'][idx]

    return theta, ustar, xi_f, uin, alpha

idx = 0
theta, ustar, xi_f, uin, alpha = load_dataX(idx)

In [3]:
num_frames = theta.shape[0]
output_path = "./"
output_gif_path = os.path.join(output_path, "data.gif")

min_theta = np.min(theta)
max_theta = np.max(theta)

min_ustar = np.min(ustar)
max_ustar = np.max(ustar)

min_xi_f = np.min(xi_f)
max_xi_f = np.max(xi_f)

def create_frame(frame):
    fig, axs = plt.subplots(3, 1, figsize=(10, 8))
    
    im0 = axs[0].imshow(theta[frame].T, cmap='inferno', vmin=min_theta, vmax=max_theta)
    axs[0].set_title('theta')
    plt.colorbar(im0, ax=axs[0], orientation='horizontal')

    im1 = axs[1].imshow(ustar[frame].T, cmap='viridis', vmin=min_ustar, vmax=max_ustar)
    axs[1].set_title('ustar')
    plt.colorbar(im1, ax=axs[1], orientation='horizontal')

    im2 = axs[2].imshow(xi_f[frame].T, vmin=min_xi_f, vmax=max_xi_f, cmap='cividis')
    axs[2].set_title('xi_f')
    plt.colorbar(im2, ax=axs[2], orientation='horizontal')
    
    for ax in axs:
        ax.axis('off')
        
    plt.tight_layout()
    filename = f"temp_frame_{frame}.png"
    plt.savefig(filename)
    plt.close(fig)
    
    return filename

with ProcessPoolExecutor() as executor:
    frame_files = list(tqdm(executor.map(create_frame, range(num_frames)),total=num_frames))

frames = [imageio.imread(frame_file) for frame_file in frame_files]
imageio.mimsave(output_gif_path, frames, duration=0.1, loop=0)

for frame_file in frame_files:
    os.remove(frame_file)

print(f"Animation saved to {output_gif_path}")

  0%|          | 0/150 [00:00<?, ?it/s]

Animation saved to ./data.gif


In [4]:
from IPython.display import HTML
HTML('<img src="./data.gif" />')